In [2]:
import numpy as np
import os

# data manager and analysis
import vodex as vx
import numan as nu

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-rnpwak1y because the default path (/home/ply/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


### Packages and versions when this notebook was last tested:

In [4]:
pip list

Package             Version   Editable project location
------------------- --------- -------------------------
asttokens           2.2.1
attrs               23.1.0
backcall            0.2.0
beautifulsoup4      4.12.2
bleach              6.0.0
comm                0.1.3
contourpy           1.0.7
cycler              0.11.0
debugpy             1.6.7
decorator           5.1.1
defusedxml          0.7.1
executing           1.2.0
fastjsonschema      2.17.1
fonttools           4.39.4
freetype-py         2.3.0
ipykernel           6.23.1
ipython             8.13.2
jedi                0.18.2
Jinja2              3.1.2
jsonschema          4.17.3
jupyter_client      8.2.0
jupyter_core        5.3.0
jupyterlab-pygments 0.2.2
kiwisolver          1.4.4
MarkupSafe          2.1.2
matplotlib          3.7.1
matplotlib-inline   0.1.6
mistune             2.0.5
nbclient            0.8.0
nbconvert           7.4.0
nbformat            5.8.0
nest-asyncio        1.5.6
numan               1.0.4     /home/ply/repos/n

# Project structure: 

Provide the project folder, then the "processed" folder will be created inside. 

As you keep going with the analysis, the folder will have the following structure: 

```
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
+++++++++++++ WILL BE DONE in this notebook +++++++++++++++
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
processed                                               +++
│   experiment_raw.json <-------------------------------+++ the file that contains the volume information about the experiment, you are creating it once and will be reusing ever after
│   experiment_dff.json <-------------------------------+++ the experiment, but files are loaded from the dff movie folder, not from the raw data               
└───dff_movie  <----------------------------------------+++ the dff movie folder, will be created by this notebook
│   │   dff_movie_0000.tif                              +++ individual tif files with the dff movie
│   │   dff_movie_0001.tif                              +++
│   │   ...                                             +++
│++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
│++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
│++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
│
└───tscore_volumes  <-------------------------------------- WILL BE DONE in 02
│   │   tscore_SvB.tif <-----------------------------------
└───spots
│   └───imaris  <------------------------------------------ Manual Step: export the spots from Imaris
│       │   └───tscore_SvB_Statistics
│       │       │     tscore_SvB_Position.csv
│       │       │     tscore_SvB_Diameter.csv
│       │       │     ...
│   └───signals  <----------------------------------------- WILL BE DONE in 03
│       │   spots_SvB.json
│   └───reports  <-----------------------------------------   
│       └───all_significant  <-----------------------------    
│           │   └───signals  <-----------------------------   
│           │       │     ...
│           │   └───images <-------------------------------    
│           │       │     ...
│       └───groupped  <------------------------------------    
│           │   readme.txt  <------------------------------ Manual Step: export the spots from Imaris
│           │   └───signals  <-----------------------------    
│           │       │     ...
│           │   └───images  <------------------------------    
│           │       │     ...
```
# Set project folder
Also, if the processed folder exists, it will complain ...

In [3]:
# please provedi FULL, not relative, path to the folder 
project_folder = "/home/ply/repos/numan/notebooks/vodex_si/data/"
project = nu.Project(project_folder)
project.create("processed")

We will also set the processed directory as our working directory, this step is important , since all the paths later are relative to this folder. Verify that the output of the cell is the "processed" folder inside your project folder. 

In [4]:
project.activate("processed")
os.getcwd()

'/home/ply/repos/numan/notebooks/vodex_si/data/processed'

# Define experiment: 

* First you need to provide the path to the folder with the **raw data**, ```data_dir```. Note that by default the code will search for all the .tif files in that folder and will treat it as raw data. While you can exclude the unwanted files later, it is recommended that three is only raw tif data in that folder. Make sure to use either ```\\``` or ```/``` as a separator, not ```\```, as it is an escape character in python.

* provide the number of **frames per volume**

* Then you need to define the **labels**.

* Then, if your experiment has a **repeating cycle**, you will need to order labels as they appear in a cycle in and provide the corresponding duration for each label. Note that the duration is in frames, as they appear in your image data, not in seconds, not in volumes.

This will initialise experiment and output the experiment information. Read it carefully! Make sure it is all as expected! Any mistake at this step will make all the future analysis wrong.

In [14]:
# files info
data_dir = "../20230207_hb02_casper_h2bcamp7f_5dpf_2P_1v2v3v4v5_1/"

# volumes info
frames_per_volume = 67
# starting_slice is zero if the recording starts exactly at the beginning of a full volume, 
# must be set to a slice where the recording starts (zero-based), slices > 0 if the recording starts somewhere in the middle of the volume:
starting_slice = 0 

# initialise experiment
experiment = vx.Experiment.from_dir(data_dir, frames_per_volume,starting_slice, verbose=True)

Image files information :

files directory: ../20230207_hb02_casper_h2bcamp7f_5dpf_2P_1v2v3v4v5_1
files [number of frames]: 
0) 20230207_hb02_casper_h2bcamp7f_5dpf_2P_1v2v3v4v5_1_MMStack_Pos0.ome.tif [8194]
1) 20230207_hb02_casper_h2bcamp7f_5dpf_2P_1v2v3v4v5_1_MMStack_Pos0_01.ome.tif [8194]
2) 20230207_hb02_casper_h2bcamp7f_5dpf_2P_1v2v3v4v5_1_MMStack_Pos0_02.ome.tif [8194]
3) 20230207_hb02_casper_h2bcamp7f_5dpf_2P_1v2v3v4v5_1_MMStack_Pos0_03.ome.tif [8194]
4) 20230207_hb02_casper_h2bcamp7f_5dpf_2P_1v2v3v4v5_1_MMStack_Pos0_04.ome.tif [8194]
5) 20230207_hb02_casper_h2bcamp7f_5dpf_2P_1v2v3v4v5_1_MMStack_Pos0_05.ome.tif [8194]
6) 20230207_hb02_casper_h2bcamp7f_5dpf_2P_1v2v3v4v5_1_MMStack_Pos0_06.ome.tif [8194]
7) 20230207_hb02_casper_h2bcamp7f_5dpf_2P_1v2v3v4v5_1_MMStack_Pos0_07.ome.tif [8194]
8) 20230207_hb02_casper_h2bcamp7f_5dpf_2P_1v2v3v4v5_1_MMStack_Pos0_08.ome.tif [8194]
9) 20230207_hb02_casper_h2bcamp7f_5dpf_2P_1v2v3v4v5_1_MMStack_Pos0_09.ome.tif [8194]
10) 20230207_hb02_casper_h2b

**Make sure everything above is correct** and then , if it is , save it for future use.
This will create a database, for more information see https://lemonjust.github.io/vodex/db/

In [6]:
experiment.save("experiment_raw.db")

Copied 2645 of 2645 pages...


# Create dff movie

In [7]:
project.create("processed/dff_movie")

# batch size 25 takes about 30 Gb of RAM to process
batch_size = 100 # in volumes
window_size = 15 # in volumes

In [8]:
nu.Preprocess(experiment).batch_dff('dff_movie',batch_size, window_size, blur_sigma = 1, verbose=True )

/home/ply/miniconda3/envs/numan/lib/python3.11/site-packages/vodex/experiment.py:601: UserWarning: list_volumes will be removed in vodex 1.1.0 use volumes property instead.
  warnings.warn(f"list_volumes will be removed in vodex 1.1.0 use volumes property instead.")


written frames : 0 - 92, out of 1984
written frames : 93 - 178, out of 1984
written frames : 179 - 264, out of 1984
written frames : 265 - 350, out of 1984
written frames : 351 - 436, out of 1984
written frames : 437 - 522, out of 1984
written frames : 523 - 608, out of 1984
written frames : 609 - 694, out of 1984
written frames : 695 - 780, out of 1984
written frames : 781 - 866, out of 1984
written frames : 867 - 952, out of 1984
written frames : 953 - 1038, out of 1984
written frames : 1039 - 1124, out of 1984
written frames : 1125 - 1210, out of 1984
written frames : 1211 - 1296, out of 1984
written frames : 1297 - 1382, out of 1984
written frames : 1383 - 1468, out of 1984
written frames : 1469 - 1554, out of 1984
written frames : 1555 - 1640, out of 1984
written frames : 1641 - 1726, out of 1984
written frames : 1727 - 1812, out of 1984
written frames : 1813 - 1898, out of 1984
written frames : 1899 - 1983, out of 1984


### Now cerate an experiment that uses dff_movie as the data
Before we make a new experiment, let's close the databse connection of the existing one.

In [12]:
experiment.close()

In [15]:
# update files info to use the new dff_movie folder instead of the original data_dir
data_dir = "dff_movie"

# create new experiment
experiment = vx.Experiment.from_dir(data_dir, frames_per_volume, starting_slice, verbose=True)

Image files information :

files directory: dff_movie
files [number of frames]: 
0) dff_movie_0000.tif [6231]
1) dff_movie_0001.tif [5762]
2) dff_movie_0002.tif [5762]
3) dff_movie_0003.tif [5762]
4) dff_movie_0004.tif [5762]
5) dff_movie_0005.tif [5762]
6) dff_movie_0006.tif [5762]
7) dff_movie_0007.tif [5762]
8) dff_movie_0008.tif [5762]
9) dff_movie_0009.tif [5762]
10) dff_movie_0010.tif [5762]
11) dff_movie_0011.tif [5762]
12) dff_movie_0012.tif [5762]
13) dff_movie_0013.tif [5762]
14) dff_movie_0014.tif [5762]
15) dff_movie_0015.tif [5762]
16) dff_movie_0016.tif [5762]
17) dff_movie_0017.tif [5762]
18) dff_movie_0018.tif [5762]
19) dff_movie_0019.tif [5762]
20) dff_movie_0020.tif [5762]
21) dff_movie_0021.tif [5762]
22) dff_movie_0022.tif [5695]

Total frames : 132928
Volumes start on frame : 0
Total good volumes : 1984
Frames per volume : 67
Tailing frames (not a full volume , at the end) : 0



**Make sure everything except the filenames looks identical to the experiment we defined at the beginning** and then , if it does , save it for future use: 

In [16]:
experiment.save("experiment_dff.db")
experiment.close()

Copied 2643 of 2643 pages...
